# Use git with github (ssh) behind corporate proxy
> using corkscrew to tunnel ssh through http proxy
- show_tags: true
- toc: true
- branch: master
- badges: false
- comments: true
- categories: [git, WSL]
- image: images/git.png

# Configuration

![alt text](../images/git_config.png "github gitlab inside outside")


I use 2 kinds of repo. gitlab for internal/corporate projects, hosted inside my company. github for public/pet projects and as a blogging platform. 3 days a week I am inside company, 4 days a week outside.

Green lines are the natural path to collaborate. 

When outside I don't have proxy configuration or firewall, and I can directly access github. I cannot access to gitlab but I don't want to address it now, this is why it is set as a black line. (if this is really needed I have a vpn access and this is as being inside)

When inside, I use internal proxy. I can directly access gitlab. But I want to access github in a transparent way. And yes from both Windows and Linux (WSL). This is the red line.


# Setup the red line

Here is the https://stackoverflow.com/questions/21318535/how-to-setup-corkscrew-to-connect-to-github-through-draconian-proxy discussion.

1st step is to install workscrew: `sudo apt install corkscrew`

Then I create 2 ssh config files:

```bash
(base) guillaume@LL11LPC0PQARQ:~/proxy_files$ cat ssh_config_noproxy
Host github.com
IdentityFile ~/.ssh/id_rsa_gmail

Host gitlab.<mycompany>.com
IdentityFile ~/.ssh/id_rsa

(base) guillaume@LL11LPC0PQARQ:~/proxy_files$ cat ssh_config_proxy
Host github.com
ProxyCommand /usr/bin/corkscrew <my_proxy_hostname> <my_proxy_port> %h %p
IdentityFile ~/.ssh/id_rsa_gmail

Host gitlab.<mycompany>.com
IdentityFile ~/.ssh/id_rsa
```

It is just a matter of linking appropriate files when I am in or out of corporate network.

As in `my_ip.sh`:

```bash
# Set Proxy
function setproxy() {
     echo "Calling setproxy"
     export {http,https,ftp}_proxy="http://proxy_ip:80"
     export {HTTP,HTTPS,FTP}_PROXY="http://proxy_ip:80"
     #proxy for wget
     ln -sf ~/proxy_files/.wgetrc_proxy ~/.wgetrc
     #proxy for apt
     #sudo ln -sf ~/proxy_files/apt_proxy.conf /etc/apt/apt.conf.d/proxy.conf
     #proxy for conda
     ln -sf ~/proxy_files/.condarc_proxy ~/.condarc
     #proxy for git
     git config --global http.proxy http://proxy_ip:80
     ln -sf ~/proxy_files/ssh_config_proxy ~/.ssh/config
}

# Unset Proxy
function unsetproxy() {
     echo "Calling unsetproxy"
     unset {http,https,ftp}_proxy
     unset {HTTP,HTTPS,FTP}_PROXY
     #no proxy for wget
     ln -sf ~/proxy_files/.wgetrc_noproxy ~/.wgetrc
     #no proxy for apt
     #sudo rm -f /etc/apt/apt.conf.d/proxy.conf
     #no proxy for conda
     ln -sf ~/proxy_files/.condarc_noproxy ~/.condarc
     #no proxy for git
     git config --global --unset http.proxy
     ln -sf ~/proxy_files/ssh_config_noproxy ~/.ssh/config
}
```

# July-21: Update following change of entreprise network

Now that we are behing GlobalProxy protection, we get some transparent proxy (good!) with no ssh access (bad).

```bash
$  ssh -Tvv git@github.com
OpenSSH_7.6p1 Ubuntu-4ubuntu0.3, OpenSSL 1.0.2n  7 Dec 2017
debug1: Reading configuration data /home/guillaume/.ssh/config
debug1: /home/guillaume/.ssh/config line 1: Applying options for github.com
debug1: Reading configuration data /etc/ssh/ssh_config
debug1: /etc/ssh/ssh_config line 19: Applying options for *
debug1: Executing proxy command: exec /bin/nc -X 5 -x 192.168.50.202:1080 github.com 22
debug1: permanently_drop_suid: 1000
debug1: identity file /home/guillaume/.ssh/id_rsa_gmail type 0
debug1: key_load_public: No such file or directory
debug1: identity file /home/guillaume/.ssh/id_rsa_gmail-cert type -1
debug1: Local version string SSH-2.0-OpenSSH_7.6p1 Ubuntu-4ubuntu0.3
ssh_exchange_identification: Connection closed by remote host
```


First step is to get rid of [socks5 configuration](https://castorfou.github.io/guillaume_blog/blog/git-avec-proxy-socks.html) 

## Revert socks5 for git

Just by commenting ProxyCommand in ssh for github

```bash
$ cat .ssh/config
Host github.com
IdentityFile ~/.ssh/id_rsa_gmail
#ProxyCommand /bin/nc -X 5 -x 192.168.50.202:1080 %h %p

Host gitlab.michelin.com
IdentityFile ~/.ssh/id_rsa
```

Second step is to migrate remote-url from ssh to https:

## from git ssh to git https

```bash
$ git remote set-url origin https://githib.com/castorfou/mit_600.2x.git
$ git remote -v
origin  https://github.com/castorfou/mit_600.2x.git (fetch)
origin  https://github.com/castorfou/mit_600.2x.git (push)
```

This allows to fetch and pull updates
```bash
git fetch
git pull
```

Lastly to setup passwordless access to github

## Github token to access passwordless using https

From https://clarusway.com/passwordless-usage-of-private-git-repositories/


To Generate token in github:
- (profile > Settings > Developer settings > Personal access tokens)
- Generate new token
- Select repo section

Integrate into git config:
- copy token into `.git/config` remote url ( from `url = https://github.com/castorfou/guillaume_blog.git` to `url = https://mytoken@github.com/castorfou/guillaume_blog.git`)